# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [14]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [15]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [17]:
import os

price_data_path = "c:\\Users\\anika\\OneDrive\\Documents\\DSI_cohort8\\Module-Production\\production\\05_src\\data\\prices"
print(f"Contents of {price_data_path}:")
print(os.listdir(price_data_path))

Contents of c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\05_src\data\prices:
['ACN', 'ALDX', 'ALL', 'AMAL', 'AMH', 'AQMS', 'BGS', 'BLPH', 'BPMX', 'BPYPN', 'BRQS', 'BWEN', 'BWG', 'CBB', 'CGEN', 'CMCTP', 'CRMT', 'CSSE', 'EARN', 'EOLS', 'ERH', 'ESGR', 'ETJ', 'FAMI', 'FIXX', 'GAZ', 'GLADD', 'GLUU', 'GLW', 'GURE', 'INSU', 'IPWR', 'ITCB', 'KALU', 'KEY', 'LEVL', 'LH', 'MNK', 'MOH', 'MOS', 'NGD', 'NPK', 'PFG', 'QRHC', 'REG', 'REI', 'RIV', 'RTTR', 'SLRX', 'SMG', 'SPXC', 'SRE', 'SYNH', 'TEF', 'TNC', 'TSN', 'VIAC', 'WORK', 'WST', 'ZIXI']


In [18]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
price_data_path = os.path.abspath(os.path.join(os.getcwd(), PRICE_DATA))

# Find parquet files in all subdirectories (recursive)
parquet_files = glob(os.path.join(price_data_path, "**", "*.parquet"), recursive=True)
print(f"Found {len(parquet_files)} parquet files")

# Load with Dask
dd_px = dd.read_parquet(parquet_files)
print(f"Loaded Dask dataframe. Columns: {dd_px.columns.tolist()}")
print("\nSample data:")
print(dd_px.head())


Found 2045 parquet files
Loaded Dask dataframe. Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source', 'ticker', 'Year']

Sample data:
             Date   Open   High    Low  Close  Adj Close      Volume   source  \
137315 2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0  ACN.csv   
137316 2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0  ACN.csv   
137317 2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0  ACN.csv   
137318 2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0  ACN.csv   
137319 2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0  ACN.csv   

       ticker  Year  
137315    ACN  2001  
137316    ACN  2001  
137317    ACN  2001  
137318    ACN  2001  
137319    ACN  2001  


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [21]:
# Write your code below.

# Add lags for Close and Adj_Close
# Group by ticker to shift within each stock
dd_px = dd_px.set_index('ticker')
dd_feat = dd_px.assign(
    Close_lag_1 = dd_px.groupby('ticker')['Close'].shift(1),
    Adj_Close_lag_1 = dd_px.groupby('ticker')['Adj Close'].shift(1)
)

# Add returns: (Close / Close_lag_1) - 1
dd_feat = dd_feat.assign(
    Returns = dd_feat['Close'] / dd_feat['Close_lag_1'] - 1
)

# Add hi_lo_range: High - Low
dd_feat = dd_feat.assign(
    hi_lo_range = dd_feat['High'] - dd_feat['Low']
)


C:\Users\anika\AppData\Local\Temp\ipykernel_61500\3849530006.py:7: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  Close_lag_1 = dd_px.groupby('ticker')['Close'].shift(1),
C:\Users\anika\AppData\Local\Temp\ipykernel_61500\3849530006.py:8: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  Adj_Close_lag_1 = dd_px.groupby('ticker')['Adj Close'].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [24]:
# Write your code below.

# 1. Convert Dask dataframe to pandas dataframe
df_feat = dd_feat.compute()

# Reset index to make 'ticker' a column again
df_feat = df_feat.reset_index()

# 2. Add 10-day moving average of Returns
# Sort by ticker and date for correct rolling calculation
df_feat = df_feat.sort_values(['ticker', 'Date'])

# Calculate moving average
df_feat['moving_average_return'] = (
    df_feat
    .groupby('ticker', group_keys=False)
    .apply(lambda x: x['Returns'].rolling(10, min_periods=1).mean())
)

print("Conversion and moving average calculation complete!")
print(f"DataFrame shape: {df_feat.shape}")
print(f"Columns: {df_feat.columns.tolist()}")
print("\nSample (first few rows):")
print(df_feat[['ticker', 'Date', 'Returns', 'moving_average_return']].head(10))

Conversion and moving average calculation complete!
DataFrame shape: (239659, 15)
Columns: ['ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'Returns', 'hi_lo_range', 'moving_average_return']

Sample (first few rows):
  ticker       Date   Returns  moving_average_return
0    ACN 2001-07-19       NaN                    NaN
1    ACN 2001-07-20 -0.010547              -0.010547
2    ACN 2001-07-23 -0.000666              -0.005607
3    ACN 2001-07-24 -0.009333              -0.006849
4    ACN 2001-07-25  0.006057              -0.003623
5    ACN 2001-07-26 -0.030100              -0.008918
6    ACN 2001-07-27  0.000690              -0.007317
7    ACN 2001-07-30  0.013094              -0.004401
8    ACN 2001-07-31  0.017687              -0.001640
9    ACN 2001-08-01  0.036096               0.002553


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

   I found I had to convert to pandas because Dask's .rolling() was giving me trouble with the grouped time-series. When I tried doing it in Dask, I got warnings about metadata and the calculation didn't work right. Pandas handled the grouped rolling window much more smoothly.

+ Would it have been better to do it in Dask? Why?

   For this assignment, pandas was actually better. Our data isn't huge, so pandas could handle it fine in memory. Dask is really for when you have data too big for your RAM. Plus, I could check my work easier in pandas - if the moving average looked wrong, I could spot it right away. Dask would be better if we had gigabytes of data or needed to process it in parallel.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.